In [1]:
import os

In [2]:
%pwd

'c:\\Users\\sandeep.sathunuri\\OneDrive - Amnet Digital\\Desktop\\machine learning\\wine-quality-pridiction\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\sandeep.sathunuri\\OneDrive - Amnet Digital\\Desktop\\machine learning\\wine-quality-pridiction'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
 from mlProject.constants import *
 from mlProject.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        print(f"📍 Reading config from: {config_filepath}")
        self.config = read_yaml(config_filepath)
        print(f"✅ config loaded: {type(self.config)}")

        self.params = read_yaml(params_filepath)
        print(f"✅ params loaded: {type(self.params)}")

        self.schema = read_yaml(schema_filepath)
        print(f"✅ schema loaded: {type(self.schema)}")
        print(f"📄 schema content: {self.schema}")

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from mlProject.utils import logger
from mlProject.utils.common import get_size

In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

📍 Reading config from: config\config.yaml
[2025-06-11 07:18:11,237: INFO: common: yaml file: config\config.yaml loaded successfully]
🔍 Content type: <class 'dict'>
📄 Content: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/SandeepSathunuri/wine-quality-pridiction/raw/refs/heads/main/archive%20(2).zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
✅ config loaded: <class 'box.config_box.ConfigBox'>
[2025-06-11 07:18:11,237: INFO: common: yaml file: params.yaml loaded successfully]
🔍 Content type: <class 'dict'>
📄 Content: {'ElasticNet': {'alpha': 0.2, 'l1_ratio': 0.1}}
✅ params loaded: <class 'box.config_box.ConfigBox'>
[2025-06-11 07:18:11,246: INFO: common: yaml file: schema.yaml loaded successfully]
🔍 Content type: <class 'dict'>
📄 Content: {'COLUMNS': {'fixed acidity': 'float64', 'volatile acidity': 'float64', 'citric acid': 'float64', 'residual sugar': '